# Data preparation

### Setup

In [1]:
# Imports
from tqdm import tqdm
import requests
import pandas as pd
import json
import os
import sys

sys.path.append('src')
import process_data
import analyse_data

## Querying data

This section contains database queries and the handling of them.

### OrthoDB

API-documentation: https://www.ezlab.org/orthodb_userguide.html#api

Ortho DB groups:
- [histone H1.0](https://www.orthodb.org/?query=19111at9604)
    - 5363206at2759 at Eukaryota level
    - 2491618at33208 at Metazoa level
    - 1590200at7742 at Vertebrata level
    - 1033145at32523 at Tetrapoda level
    - 21759at40674 at Mammalia level
    - 97195at9347 at Eutheria level
    - 97195at314146 at Euarchontoglires level
    - 97195at9443 at Primates level
    - 19111at314295 at Hominoidea level (1)
    - 19111at9604 at Hominidae level (2)

In [4]:
# Setting Ortho DB group to query
group = '5363206at2759'

# Defining filename for output
filename = 'exp/ortho_h1-0/H1-0_orthodb.json'

In [5]:
# Querying API
url = f"https://data.orthodb.org/current/orthologs?id={group}"
response = requests.get(url)
data = response.json()

In [6]:
# Saving to metadata
with open(filename, 'w') as file:
    json.dump(data, file, indent=4)

## Getting template metadata

This section uses general code blocks to prepare template metadata in experiment .json files.

Fetching metadata requires a starting metadata .json file containing UniProt IDs (See `~/exp/README.md`).
Custom IDs can be arbitrary, and may be reassigned during the following procedures.

#### How to get template metadata
1. **Specify experiment**
2. **Specify templates**
3. **Get metadata**
4. **Format custom protein IDs** (_Optional_)
5. **Save metadata**

#### 1. Experiment name

In [7]:
experiment = 'initial'

In [29]:
experiment = 'ortho_h1-0'

#### 2. Specify templates

In [22]:
# Getting template UniProt IDs by experiment-specific methods

if experiment == 'initial':
    # Manual entries
    uids = ['P07305', 'B4DR52']
    ids = ['H1-0', 'H2B']

elif experiment == 'ortho_h1-0':
    # Entries from OrthoDB
    with open('exp/ortho/H1-0_orthodb.json') as file:
        data = json.load(file)
    uids = []
    no_uids = []
    for id in range(len(data['data'])):
        try:
            for protein in data['data'][id]['genes']:
                uids.append(protein['uniprot']['id'])
        except KeyError:
            no_uids.append(data['data'][id]['organism']['name']) # Skipping species without UniProt entries
    if no_uids:
        print(f"The following {len(no_uids)}/{len(data['data'])} entries had no UniProt IDs")
        print(*[f" - {specie}" for specie in no_uids], sep="\n")
    ids = list(range(len(uids)))

# Loading into metadata with arbitrary custom IDs
metadata_path = f'exp/{experiment}/{experiment}.json'
metadata = {'templates': {id: {"uniprot_id": uniprot_id} for uniprot_id, id in zip(uids, ids)}}
with open(metadata_path, 'w') as file:
    json.dump(metadata, file, indent=4)

The following 319/485 entries had no UniProt IDs
 - Cyprinodon tularosa
 - Gambusia affinis
 - Girardinichthys multiradiatus
 - Nematolebias whitei
 - Poecilia formosa
 - Poecilia reticulata
 - Xiphophorus hellerii
 - Archocentrus centrarchus
 - Simochromis diagramma
 - Acanthopagrus latus
 - Alosa alosa
 - Alosa sapidissima
 - Anarrhichthys ocellatus
 - Anguilla anguilla
 - Boleophthalmus pectinirostris
 - Cheilinus undulatus
 - Chelmon rostratus
 - Colossoma macropomum
 - Coregonus clupeaformis
 - Cyclopterus lumpus
 - Cyprinus carpio
 - Denticeps clupeoides
 - Echeneis naucrates
 - Electrophorus electricus
 - Epinephelus lanceolatus
 - Erpetoichthys calabaricus
 - Etheostoma cragini
 - Etheostoma spectabile
 - Gadus morhua
 - Gasterosteus aculeatus aculeatus
 - Gouania willdenowi
 - Hippoglossus hippoglossus
 - Hippoglossus stenolepis
 - Hypomesus transpacificus
 - Lates calcarifer
 - Lepisosteus oculatus
 - Megalobrama amblycephala
 - Megalops cyprinoides
 - Micropterus dolomieu
 -

#### 3. Get template metadata

In [23]:
# Getting experiment json
with open(metadata_path, 'r') as file:
    metadata = json.load(file)

# Getting names, descriptions, and species
deleted_records = []
for id, fields in tqdm(metadata['templates'].items()):
    uniprot_id = fields['uniprot_id']
    try:
        name, desc, spec, seq = process_data.get_protein_metadata(uniprot_id)
    except ValueError:
        deleted_records.append(id)
        continue
    metadata['templates'][id] = {'uniprot_id': uniprot_id, 'name': name, 'description': desc, 'species': spec, 'sequence': seq}

# Discarding deleted UniProt records
if deleted_records:
    print(f"The UniProt IDs returned no records:")
    for id in deleted_records:
        print(f" - {metadata['templates'][id]['uniprot_id']}")
        del metadata['templates'][id]

pd.DataFrame(metadata['templates']).transpose()

100%|██████████| 199/199 [01:18<00:00,  2.55it/s]

The UniProt IDs returned no records:
 - A0A5F4D9L8
 - G3W6X3
 - A0A1L8GP11


,uniprot_id,name,description,species,sequence
0,A0A2I4B7C3,A0A2I4B7C3_9TELE,histone H1.0-B,Austrofundulus limnaeus,MAETSSAPSKAKRAAKPKKPASHPKYSDMIKAAIVHDASRSGASRQ...
1,A0A3Q2C803,A0A3Q2C803_CYPVA,Histone H1.0-B,Cyprinodon variegatus (Sheepshead minnow),MAETAAPAKAKRASKPKKPTSHPKYSDMIKAAIVHDASRSGASRQS...
2,A0A3Q2Q6G9,A0A3Q2Q6G9_FUNHE,Histone H1.0-B,Fundulus heteroclitus (Killifish),MAETSAAPAKAKRASKPKKPASHPKYSDMIKAAIVHDASRTGASRQ...
3,A0A3Q3AJ14,A0A3Q3AJ14_KRYMA,Histone H1.0-B,Kryptolebias marmoratus (Mangrove killifish),MAETSTAPAKAKRASKPKKPASHPKYSEMIKAAIVHDASRSGASRQ...
4,A0A1A8V9N7,A0A1A8V9N7_NOTFU,"H1 histone family, member 0",Nothobranchius furzeri (Turquoise killifish),MAETSGAPAKAKKASKPKKPASHPKYSDMIKAAIVHDASRSGASRQ...
...,...,...,...,...,...
194,W2TEX0,W2TEX0_NECAM,Linker histone H1 and H5 family protein,Necator americanus (Human hookworm),MMSVAAVSSSKRPPKATALRVPKSMPEHPTYAVMVKDAITQLKERS...
195,A0A090KXU4,A0A090KXU4_STRRB,Histone H1.0,Strongyloides ratti (Parasitic roundworm),MTTDVTVQPAAAGTSTKKASTSAPKSVKKSSHPVYADMVKKAIIEL...
196,A0A0D2VKG3,A0A0D2VKG3_CAPO3,H15 domain-containing protein,Capsaspora owczarzaki (strain ATCC 30864),MSTTAAAAAPPAPKANKPKKVPIHPPYNVMVLEAIKELGDRSGSSR...
197,P15870,H1D_STRPU,Histone H1-delta,Strongylocentrotus purpuratus (Purple sea urchin),MADTDAAPAAPAPSTPKKAAKKKASKPKTPASHPKYSDMIASALES...


#### 4. Format custom protein IDs _(Optional)_

In [24]:
# Setting IDs from UniProt name
templates = {record['name'].split('_')[1]: record for record in metadata['templates'].values()}

In [25]:
# Setting IDs from integer range and UniProt name
templates = {str(i) + '_' + record['name'].split('_')[1]: record for record, i in zip(metadata['templates'].values(), range(len(metadata['templates'])))}

In [26]:
# Display keys
if len(metadata['templates']) != len(templates):
    print("IDs are not unique!")
templates.keys()

dict_keys(['0_9TELE', '1_CYPVA', '2_FUNHE', '3_KRYMA', '4_NOTFU', '5_9TELE', '6_9TELE', '7_XIPMA', '8_HAPBU', '9_9CICH', '10_NEOBR', '11_OREAU', '12_9CICH', '13_9TELE', '14_ACIRT', '15_ACIRT', '16_AMPOC', '17_ANATE', '18_ASTMX', '19_BETSP', '20_CARAU', '21_CARAU', '22_CARAU', '23_CHACN', '24_CLUHA', '25_CYNSE', '26_DANRE', '27_ESOLU', '28_GYMAC', '29_HIPCM', '30_ICTPU', '31_9LABR', '32_9TELE', '33_9TELE', '34_ONCMY', '35_ONCMY', '36_ORYLA', '37_ORYME', '38_9TELE', '39_9TELE', '40_PERFV', '41_PERFL', '42_PYGNA', '43_SALFA', '44_SALSA', '45_SALSA', '46_SALTR', '47_SALTR', '48_SCOMX', '49_SERDU', '50_SERLL', '51_9TELE', '52_SINGR', '53_9TELE', '54_9TELE', '55_9TELE', '56_9TELE', '57_FICAL', '58_9PASS', '59_9PASS', '60_TAEGU', '61_AYTFU', '62_CHICK', '63_EGRGA', '64_APTFO', '65_AQUCH', '66_PYGAD', '67_STRHB', '68_STRCA', '69_9SAUR', '70_PANGU', '71_PODMU', '72_9SAUR', '73_PSETE', '74_9SAUR', '75_9SAUR', '76_9SAUR', '77_ALLMI', '78_ALLSI', '79_CHEMY', '80_PELSI', '81_BALAS', '82_DELLE', '83

In [27]:
# Assigning keys
# Checking that new keys are unique
assert len(metadata['templates']) == len(templates), "IDs are not unique!"

metadata['templates'] = templates

#### 5. Saving metadata

In [28]:
# Saving metadata
with open(metadata_path, 'w') as file:
    json.dump(metadata, file, indent=4)

## Filtering templates

### H1.0 orthologs

Some hits from the BLAST are irrelevant, as they are most likely not orthologs, like:
- "Uncharacterized protein"
- "8-amino-7-oxononanoate synthase-like"

Additionally, some hits are more subtle orthologs, like other members of the H15 superfamily:
- "H5 protein"
- "H15 domain-containing protein"

Lastly, some are in a grey area of  the H1.0 definition:
- "histone H1.0-like"
- "Histone H1.0-B"
- "histone H1.0-B-like"
- "H1 histone family, member 0"

This section is used to specify the names of the orthologs to be used for analysis

In [30]:
# Loading metadata
metadata_path = "exp/ortho_h1-0/ortho_h1-0.json"
with open(metadata_path, 'r') as file:
    metadata = json.load(file)['templates']

templates = pd.DataFrame(metadata).transpose()

In [31]:
# Finding all unique descriptions / names
templates['description'].value_counts()

histone H1.0                               38
Histone H1.0                               29
H1.0 linker histone                        28
Histone H1.0-B                             22
H15 domain-containing protein              17
histone H1.0-like                          12
Histone H5                                 11
histone H1.0-B                              6
Histone H1.0-B-like                         6
Linker histone H1 and H5 family protein     5
Histone H1.0-like                           3
histone H5                                  3
histone H1.0-B-like                         2
late histone H1-like                        2
H1 histone family, member 0 like 1          1
H1 histone family member 0 like protein     1
8-amino-7-oxononanoate synthase-like        1
Putative histone h1.0                       1
Uncharacterized protein                     1
H1F0 isoform 1                              1
Histone H1-beta, late embryonic             1
Histone H1                        

In [32]:
# Setting description requirements
required  = ['H1', 'h1', 'H5', 'h5']

In [33]:
# Filtering off
filtered = templates[templates.apply(lambda row: max([req in row['description'] for req in required]), axis=1)]

# Displaying proteins that were filtered off
templates.loc[list(set(templates.index) - set(filtered.index))]

,uniprot_id,name,description,species,sequence
108_SHEEP,A0A6P3E3Z9,A0A6P3E3Z9_SHEEP,Uncharacterized protein,Ovis aries (Sheep),MTENSTSTPAAKPKRAKASKKSTDHPKYSDMIVAAIQAEKNRAGSS...
74_9SAUR,A0A6I9XPX0,A0A6I9XPX0_9SAUR,8-amino-7-oxononanoate synthase-like,Thamnophis sirtalis,MWCSQFFRSPFVLGQCSPSRAQSALAQLKHLLEGELEGIRGAGTWK...
187_CAEBR,A8Y2Z3,A8Y2Z3_CAEBR,Protein CBG22701,Caenorhabditis briggsae,MHCFAITPTPEGSPAPEDSPVAAQTTPKRPAATPAKARKAKRGPGR...


In [34]:
# Saving to metadata
metadata['templates'] = filtered.to_dict(orient='index')
with open(metadata_path, 'w') as file:
    json.dump(metadata, file, indent=4)

## Generating sequences for simulations

This section uses general code blocks to assemble IDR sequences and save them as FASTA files and metadata .json files.

Generating data requires a starting metadata .json file containing UniProt IDs (See `~/exp/README.md`)

#### How to prepare data
1. **Specify experiment**
2. **Specify variants**
3. **Specify IDR region**
4. **Generate data**
5. **Save sequences**
6. **Save metadata**

### 1. Experiment name

In [24]:
experiment = 'initial'

In [13]:
experiment = 'ortho_h1-0'

### 2. Variant types

In [18]:
# All
variants = list(process_data.variant_types.keys())

In [19]:
# Only wildtype
variants = 'wt'

### 3. IDR region

In [20]:
# CTD
i_idr=-1
length_order=False
region_filter = ['CTD']

In [ ]:
# NTD
i_idr=0
length_order=True
region_filter = ['NTD']

In [ ]:
# Longest IDR
i_idr=0
length_order=True
region_filter = ['NTD', 'INT', 'CTD']

### 4. Generating data

In [33]:
# Extracting IDRs
# Getting experiment json
metadata_path = f'exp/{experiment}/{experiment}.json'
with open(metadata_path, 'r') as file:
    metadata = json.load(file)

# Initiating DataFrame
proteins = []
no_IDP = []
for id, fields in tqdm(metadata['templates'].items()):
    try:
        seq, loc, reg = process_data.get_protein_idr(uniprot_id=fields['uniprot_id'], i_idr=i_idr, length_order=length_order)
    except ValueError:
        no_IDP.append((id, fields['uniprot_id']))
        continue
    proteins.append({'template': id, 'sequence': seq, 'location': loc, 'region': reg})
if no_IDP:
    print(f"The following {len(not_IDP)} UniProt IDs did not return IDPs:")
    print(*[f" - {uid}\t({id})" for id, uid in not_IDP], sep='\n')
data = pd.DataFrame(proteins)

# Filtering according to CTD/NTD settings
wrong_region = data[~data['region'].isin(region_filter)]
if len(wrong_region) > 0:
    print(f"The following UniProt IDs had no {region_filter[0]} IDR")
    print(*[f" - {metadata['templates'][i]['uniprot_id']}" for i in wrong_region['template']],sep='\n')
data = data[data['region'].isin(region_filter)]

# Removing fragment sequences (contains amino acid 'X')
fragments = data[data['sequence'].str.contains('X')]
if len(fragments) > 0:
    print("The following UniProt IDs were fragments containing 'X' in their sequence")
    print(*[f" - {metadata['templates'][i]['uniprot_id']}" for i in fragments['template']],sep='\n')
data = data[~data['sequence'].str.contains('X')]

# Expanding dataframe to include all variants
data['variant'] = [variants]*len(data)
if type(variants) == list:
    data = data.explode('variant')
    data.index = data['template'] + '_' + data['variant'].str.upper()
else:
    data.index = data['template']

# Generating variant sequence
data['sequence'] = data.apply(lambda row: process_data.generate_variant(row['sequence'], row['variant']), axis=1)

# Setting variant description
data['variant'] = data.apply(lambda row: process_data.variant_types[row['variant']]['name'], axis=1)

data

100%|██████████| 193/193 [01:16<00:00,  2.52it/s]

The following 1 UniProt IDs did not return IDPs:
 - A0A1L8GH50	(172_XENLA)
The following UniProt IDs had no CTD IDR
 - W2TL48
The following UniProt IDs were fragments containing 'X' in their sequence
 - A0A2R2MRR7


,template,sequence,location,region,variant
template,,,,,
0_9TELE,0_9TELE,HTKGIGASGSFRLTKPEDSSKKPAKAAAAVAAKPKKVAKAKPKKAA...,80:197,CTD,Wild type
1_CYPVA,1_CYPVA,RHTKGIGASGSFRLTKPEDTKKPAAKAASAPKPKKVVKPKPKKAAK...,78:194,CTD,Wild type
2_FUNHE,2_FUNHE,RHTKGIGASGSFRLTKPEDTKKTPAKAASAAKPRKVAKPKPKKAAK...,79:195,CTD,Wild type
3_KRYMA,3_KRYMA,HTKGIGASGSFRLTKPEDSSKKPAKAAAAAASAKPKKVAKPKPKKA...,80:198,CTD,Wild type
4_NOTFU,4_NOTFU,RHTKGIGASGSFRLTKPEDSKKPAKAAASAKPKKVAKPKPKKAAKP...,79:194,CTD,Wild type
...,...,...,...,...,...
191_NECAM,191_NECAM,RKEVVRKEAAQKAKPTRNTTGSSSRKTVKSVKKSTASRV,168:207,CTD,Wild type
192_STRRB,192_STRRB,KKSSEAKKVTDSKKSTDVKKTPETKKAVKAPKAKATEKKVLKKAIV...,138:221,CTD,Wild type
193_CAPO3,193_CAPO3,AAALPKAAKAPKAAKAPKAEKPAKAAKAPKADKPAKAAKAPKAAKA...,213:355,CTD,Wild type


### 5. Saving sequences

In [34]:
# Saving sequences
datadir = f'exp/{experiment}/data'
os.makedirs(datadir, exist_ok=True)

# Iterating over rows
for index, cols in data.iterrows():
    id = index.upper()
    desc = cols['variant']
    seq = cols['sequence']

    # Saving as FASTA format (Though one-line sequence)
    filepath = datadir + '/' + id + '.fasta'
    with open(filepath, 'w') as file:
        file.write('>' + id + ' ' + desc + '\n')
        file.write(seq + '\n')

### 6. Saving metadata

In [35]:
# Saving metadata
metadata['data'] = data.to_dict(orient='index')
with open(metadata_path, 'w') as file:
    json.dump(metadata, file, indent=4)

## Generating average sequence

This section uses general code blocks to generate average sequences and save them as FASTA files and metadata .json files.

Generating data requires a starting metadata .json file containing template sequences to average over.

#### 1. Template sequences

In [30]:
template_experiment = 'ortho_h1-0'

#### 2. Average experiment meta data

In [31]:
avg_name = 'H1-0_AVG'

In [32]:
avg_experiment = 'idpevo_h1-0'

#### 3. Generating average sequence

In [33]:
# Getting experiment json
metadata_path = f'exp/{template_experiment}/{template_experiment}.json'
with open(metadata_path, 'r') as file:
    metadata = json.load(file)
templates = pd.DataFrame(metadata['templates']).transpose()
data = pd.DataFrame(metadata['data']).transpose()

# Generating average
avg = analyse_data.average_sequence(data['sequence'])
avg

'KEAVASAPAKKSKSKSKKVAARKKFEVRATPSVKPGKKKKASKPVKKDKAKKPAAKKKSPKATKLPKRTKKKPKGTAKPASKAPKKTAPKKAKAGKPKKVKAKATVPA'

#### 4. Saving sequence

In [34]:
# Saving sequence
file_path = f'exp/{avg_experiment}/data/{avg_name}.fasta'
with open(file_path, 'w') as file:
    file.write(">" + avg_name + "\n")
    file.write(avg + "\n")

#### 5. Saving meta data

In [ ]:
# Saving new experiment json
metadata['data'] = {avg_name: {'template': list(templates.index), 'sequence': avg, 'location': None, 'region': 'CTD', 'variant': 'Averaged'}}
metadata_path = f'exp/{avg_experiment}/{avg_experiment}.json'
with open(metadata_path, 'w') as file:
    json.dump(metadata, file, indent=4)